In [22]:
import requests
import psycopg2
import random
import folium
from folium.plugins import HeatMap

# Create synthetic Data

In [24]:
# Bereich für zufällige Breiten- und Längengrade in Berlin Kreuzberg/Friedrichshain
latitude_range = (52.49, 52.52)
longitude_range = (13.40, 13.45)

In [50]:
# Rating-Optionen
ratings = [1, 2, 3, 4, 5]

# Funktion zur Generierung zufälliger Koordinaten innerhalb des angegebenen Bereichs
def generate_random_coordinates():
    latitude = random.uniform(*latitude_range)
    longitude = random.uniform(*longitude_range)
    return latitude, longitude

# Funktion zur Generierung eines zufälligen Ratings
def generate_random_rating():
    return random.choice(ratings)

# Anzahl der Einfügungen
num_entries = 100  # Diese Zahl können Sie anpassen

# Insert Routes

In [54]:
def inseart_route(data, start_lat, start_lon, end_lat, end_lon, rating):
    # Connect to your PostgreSQL database
    conn = psycopg2.connect("dbname='bike_project' user='postgres' host='localhost' password='123456'")
    cur = conn.cursor()

    # Rating to weight adjustment mapping
    rating_weight_adjustment = {
        1: -2,
        2: -1,
        3: 0,
        4: 1,
        5: 2
    }

    # Insert route into the routes table with rating
    cur.execute("""
        INSERT INTO routes (start_latitude, start_longitude, end_latitude, end_longitude, rating) 
        VALUES (%s, %s, %s, %s, %s) RETURNING id
        """, (start_lat, start_lon, end_lat, end_lon, rating))
    route_id = cur.fetchone()[0]

    # Function to check if a node exists
    def get_node_id(latitude, longitude):
        cur.execute("SELECT id FROM nodes WHERE latitude = %s AND longitude = %s", (latitude, longitude))
        result = cur.fetchone()
        return result[0] if result else None

    # Insert nodes and route_nodes into the database
    sequence = 0
    coordinates = data['routes'][0]['geometry']['coordinates']
    #coordinates = data['routes'][0]['legs'][0]['annotation']['nodes']
    print(coordinates)

    # List to store node IDs in the order of the route
    node_ids = []

    # Loop through coordinates excluding start and end point
    for coordinate in coordinates[1:-2]:
        longitude, latitude = coordinate
        node_id = get_node_id(latitude, longitude)
        
        if not node_id:
            cur.execute("INSERT INTO nodes (latitude, longitude) VALUES (%s, %s) RETURNING id", (latitude, longitude))
            node_id = cur.fetchone()[0]
        
        cur.execute("INSERT INTO route_way (route_id, node_id, sequence) VALUES (%s, %s, %s)", (route_id, node_id, sequence))
        sequence += 1
        node_ids.append(node_id)

    # Function to check if an edge exists and return its weight
    def get_edge_weight(node_id_start, node_id_end):
        cur.execute("SELECT weight FROM edges WHERE node_id_start = %s AND node_id_end = %s", (node_id_start, node_id_end))
        result = cur.fetchone()
        return result[0] if result else None

    # Function to update edge weight
    def update_edge_weight(node_id_start, node_id_end, weight_adjustment):
        cur.execute("UPDATE edges SET weight = weight + %s WHERE node_id_start = %s AND node_id_end = %s", 
                    (weight_adjustment, node_id_start, node_id_end))

    # Insert edges into the database
    for i in range(len(node_ids) - 1):
        start_node_id = node_ids[i]
        end_node_id = node_ids[i + 1]
        weight_adjustment = rating_weight_adjustment[rating]
        
        current_weight = get_edge_weight(start_node_id, end_node_id)
        
        if current_weight is None:
            cur.execute("INSERT INTO edges (node_id_start, node_id_end, weight) VALUES (%s, %s, %s)", 
                        (start_node_id, end_node_id, weight_adjustment))
        else:
            update_edge_weight(start_node_id, end_node_id, weight_adjustment)

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

# Call random Entries

In [55]:
for i in range(num_entries):
    # Generieren der zufälligen Start- und Endkoordinaten
    start_lat, start_lon = generate_random_coordinates() # 
    end_lat, end_lon = generate_random_coordinates()
    
    # Generieren des zufälligen Ratings
    rating = generate_random_rating()

    url = f"http://router.project-osrm.org/route/v1/driving/{start_lon},{start_lat};{end_lon},{end_lat}?geometries=geojson&annotations=nodes"

    response = requests.get(url)
    data = response.json()

    inseart_route(data, start_lat, start_lon, end_lat, end_lon, rating)

    print(data)

[[13.411861, 52.493908], [13.413619, 52.493338], [13.41263, 52.492019], [13.408503, 52.493088], [13.403492, 52.493924], [13.405158, 52.496003], [13.405078, 52.49641], [13.409963, 52.502716], [13.410549, 52.503432], [13.4111, 52.503534], [13.411045, 52.504052], [13.416455, 52.511006], [13.42117, 52.509775], [13.423734, 52.513057], [13.426713, 52.515812], [13.435388, 52.514724], [13.435955, 52.51524], [13.43652, 52.515332], [13.439511, 52.514058], [13.442854, 52.51216], [13.444092, 52.511622], [13.444453, 52.511803]]
{'code': 'Ok', 'routes': [{'geometry': {'coordinates': [[13.411861, 52.493908], [13.413619, 52.493338], [13.41263, 52.492019], [13.408503, 52.493088], [13.403492, 52.493924], [13.405158, 52.496003], [13.405078, 52.49641], [13.409963, 52.502716], [13.410549, 52.503432], [13.4111, 52.503534], [13.411045, 52.504052], [13.416455, 52.511006], [13.42117, 52.509775], [13.423734, 52.513057], [13.426713, 52.515812], [13.435388, 52.514724], [13.435955, 52.51524], [13.43652, 52.515332]

# Map Creation

In [56]:
# Funktion zur Bestimmung der Farbe basierend auf dem Gewicht
def get_color(weight):
    if weight < 0:
        return 'red'
    elif weight > 0:
        return 'green'
    else:
        return 'yellow'

# Verbindung zur PostgreSQL-Datenbank
conn = psycopg2.connect("dbname='bike_project' user='postgres' host='localhost' password='123456'")
cur = conn.cursor()

# Abfrage aller Edges mit deren Start- und Endkoordinaten sowie Gewichten
cur.execute("""
    SELECT ns.latitude, ns.longitude, ne.latitude, ne.longitude, e.weight
    FROM edges e
    JOIN nodes ns ON e.node_id_start = ns.id
    JOIN nodes ne ON e.node_id_end = ne.id
""")
edges = cur.fetchall()

# Karte initialisieren
map_center = [52.5, 13.4]  # Beispielzentrum (Berlin)
mymap = folium.Map(location=map_center, zoom_start=12)

# Edges zur Karte hinzufügen
for edge in edges:
    start_lat, start_lon, end_lat, end_lon, weight = edge
    color = get_color(weight)
    folium.PolyLine(
        locations=[(start_lat, start_lon), (end_lat, end_lon)],
        color=color,
        weight=5,
        opacity=0.8
    ).add_to(mymap)

# HTML-Datei speichern
mymap.save('route_map.html')

# Verbindung schließen
cur.close()
conn.close()
